In [1]:
import gc
import glob
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import utils
import utils_fe
from gbm_pipeline import GBMPipeline
from tqdm import tqdm

/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


Using TensorFlow backend.


In [3]:
src_oof = 'oof/'
src_preds = '/home/w/Projects/Toxic/scripts/predictions/'
src_features = '../data/features/'

run_kfold = True
split_for_validation = True
add_features = True

run_kfold = 1


if run_kfold:
    prefix = 'KFold'
else:
    prefix = 'Bag'
    
run_name = 'LGB_{}_StackerNext_Params1'.format(prefix),

In [4]:
train = pd.read_pickle("../data/train_basic_clean.pkl")
test = pd.read_pickle("../data/test_basic_clean.pkl")
target_columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

oof_tr, oof_te = utils.load_predictions(src_oof, contains='')
oof_tr, oof_te = np.asarray(oof_tr), np.asarray(oof_te)
print(oof_tr.shape, oof_te.shape)

X_train_oof = pd.DataFrame()
X_test_oof = pd.DataFrame()

for i in range(oof_tr.shape[0]):
    oof_tr_part = oof_tr[i][:, :, 0]
    oof_te_part = oof_te[i].mean(axis=-1)[:, :]
    X_train_oof = pd.concat([X_train_oof, pd.DataFrame(oof_tr_part)], axis=1)
    X_test_oof = pd.concat([X_test_oof, pd.DataFrame(oof_te_part)], axis=1)


f_cols = []

for i in range((X_train_oof.shape[1] // 6)):
    f_cols.extend(['{}_preds_model{}'.format(target_columns[x], i) for x in range(len(target_columns))])

X_train_oof.columns = f_cols
X_test_oof.columns = f_cols


if add_features:
    data_tokenized = pd.read_pickle(src_features + 'data_TokenizedSentences160.pkl')
    data_badwords300 = pd.read_pickle(src_features + 'data_Binary300Badwords.pkl')
    data_textStatistics = pd.read_pickle(src_features + 'data_TextStatistics.pkl')
    data_transformations = pd.read_pickle(src_features + 'data_20dim_SVDLSA.pkl')

    data_tokenized.reset_index(drop=True, inplace=True)
    
    X = pd.concat([data_tokenized, data_textStatistics, data_transformations], axis=1)
    X['badwordsCount'] = data_badwords300.sum(axis=1)

    X_train = X.iloc[:train.shape[0], :]
    X_test = X.iloc[train.shape[0]:, :]
    y_train = train[target_columns].values
    
    X_test.reset_index(drop=True, inplace=True)
    features = np.setdiff1d(X_train.columns, target_columns)

    X_train = pd.concat([X_train[features], X_train_oof], axis=1)
    X_test = pd.concat([X_test[features], X_test_oof], axis=1)
    
    del X, test
    del data_tokenized, data_badwords300
    del data_textStatistics, data_transformations
    gc.collect()

else:
    X_train = X_train_oof
    y_train = train[target_columns].values
    X_test = X_test_oof
    

print(X_train.shape, X_test.shape)


Train files:

['oof/train/Conv2Dmodel128_KF3_KFold1_FastText2_BasicClean2_300dim_200k_320len_random0_5fold_BS256_Nadam0.001_Stratified_27.31593.pkl',
 'oof/train/Conv2Dmodel128_KF3_KFold1_FastText2_BasicClean2_300dim_200k_320len_random0_5fold_BS256_Nadam0.001_Stratified_fromCheck_27.31593.pkl',
 'oof/train/GRUCapsuleBidirectional128_KF2_KFold1_FastText2_BasicClean2_300dim_200k_320len_random0_5fold_BS256_Adam0.001_Stratified_22.37225.pkl',
 'oof/train/GRUHierarchicalBidirectional128_KF3_KFold1_FastText2_BasicClean2_300dim_200k_320len_random0_5fold_BS256_Adam0.001_Stratified_23.81552.pkl',
 'oof/train/GRUconcat2Bidirectional128_KF2_KFold1_FastText2_BasicClean2_300dim_200k_320len_random0_5fold_BS256_Adam0.001_Stratified_21.74320.pkl',
 'oof/train/GRUconcatBidirectional256_1stSet_KFold1_Glove_BasicClean_200dim_300k_256len_random0_5fold_BS512_Nadam_94.57426.pkl',
 'oof/train/GRUconcatBidirectional256_1stSet_KFold1_Glove_BasicClean_200dim_300k_256len_random0_5fold_BS512_Nadam_fromCheck_94.5

In [5]:
xgb_params = {
    'objective': 'binary:logloss',
    'eta': 0.05,
    'max_depth': 8,
    'min_child_weight': 1,
    'colsample_bytree': 0.7,
    'scale_pos_weight': 1,
    'subsample': 0.8,
    'lambda': 0,
    'tree_method': 'hist',
    'nthread': 4,
    'silent': True,
    'base_score': 0.0,
}

# Best Recruit's parameters
lgb_params1 = {
    'learning_rate': 0.05,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'sub_feature': 0.7,
    'num_leaves': 60,
    'min_data': 100,
    'min_hessian': 1,
    'verbose': -1,
    'num_threads': 10,
}

# Best Favorita parameters
lgb_params2 = {
    'learning_rate': 0.05,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'min_data_in_leaf': 200,
    'num_leaves': 80,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.7,
    'bagging_freq': 1,
    'num_threads': 10,
}


if split_for_validation:
    
    train_params = {
        'boost_round': 10000,
        'stopping_rounds': 50,
        'verbose_eval': 50,
    }
    
else:
    
    train_params = {
        'boost_round': 161,
        'stopping_rounds': 50,
        'verbose_eval': False,
    }

    
pipeline_params = {
    'use_lgb': True,
    'predict_test': True,
    'seed': 1337,
    'shuffle': True,
    'verbose': True,
    'run_save_name': run_name,
    'save_model': False,
    'save_history': False,
    'save_statistics': True,
    'output_statistics': True,
    'output_importance': True,
}


XGB_pipeline = GBMPipeline(
    use_lgb=pipeline_params['use_lgb'],
    predict_test=pipeline_params['predict_test'],
    seed=pipeline_params['seed'],
    shuffle=pipeline_params['shuffle'],
    verbose=pipeline_params['verbose'],
    run_save_name=pipeline_params['run_save_name'],
    save_model=pipeline_params['save_model'],
    save_history=pipeline_params['save_history'],
    save_statistics=pipeline_params['save_statistics'],
    output_statistics=pipeline_params['output_statistics'],
    output_importance=pipeline_params['output_importance'],
)


In [6]:
np.setdiff1d(X_train.columns, X_test.columns)

array([], dtype=object)

In [8]:
if pipeline_params['use_lgb']:
    gbm_params = lgb_params1
else:
    gbm_params = xgb_params
    
if not add_features:
    features = None


if pipeline_params['predict_test']:
    val_preds, test_preds, gbm = XGB_pipeline.fold_run(X_train, train[target_columns],
                                           X_test=X_test,
                                           model_params=gbm_params,
                                           train_params=train_params,
                                           output_submission=True,
                                           additional_features=features,
                                           save_oof=True,
                                           stratify=True)
else:
    val_preds, gbm = XGB_pipeline.fold_run(X_train[features], y_train=X_train[targets],
                               model_params=gbm_params,
                               train_params=train_params)


utils.save_parameter_dict(
    'checkpoints/{0}/{0}_gbm_parameters.txt'.format(pipeline_params['run_save_name']), gbm_params)
utils.save_parameter_dict('checkpoints/{0}/{0}_train_parameters.txt'.format(
    pipeline_params['run_save_name']), train_params)
utils.save_parameter_dict('checkpoints/{0}/{0}_pipeline_parameters.txt'.format(
    pipeline_params['run_save_name']), pipeline_params)

Using LightGBM
OOF train predictions shape: (159571, 6)
X_train shape: (159571, 358)
OOF test predictions shape: (153164, 6, 5)
X_test shape: (153164, 358)
Running KFold run with 5 folds
Start training with parameters: {'learning_rate': 0.05, 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 'sub_feature': 0.7, 'num_leaves': 60, 'min_data': 100, 'min_hessian': 1, 'verbose': -1, 'num_threads': 10} 
 

Training model for column: toxic
(127655, 248) (31916, 248) (153164, 248)
Training until validation scores don't improve for 50 rounds.
[50]	valid_0's auc: 0.985506
[100]	valid_0's auc: 0.98571
[150]	valid_0's auc: 0.986067
[200]	valid_0's auc: 0.985978
Early stopping, best iteration is:
[153]	valid_0's auc: 0.986073
Minimum validation split loss for current fold/bag: 0.9860734831568813 

Seconds it took to train the model: 41.078657388687134 

Best iterations: [153, 24, 153] 

Visualize model feature importance.
Predicting on test data.
Training model for column: severe_tox

[50]	valid_0's auc: 0.976045
Early stopping, best iteration is:
[28]	valid_0's auc: 0.976511
Minimum validation split loss for current fold/bag: 0.976510842405677 

Seconds it took to train the model: 147.4505820274353 

Best iterations: [153, 24, 153, 24, 82, 16, 89, 39, 119, 19, 92, 53, 89, 80, 111, 29, 91, 28] 

Visualize model feature importance.
Predicting on test data.
Training model for column: insult
(127657, 248) (31914, 248) (153164, 248)
Training until validation scores don't improve for 50 rounds.
[50]	valid_0's auc: 0.988242
[100]	valid_0's auc: 0.989028
Early stopping, best iteration is:
[90]	valid_0's auc: 0.989045
Minimum validation split loss for current fold/bag: 0.9890447787754671 

Seconds it took to train the model: 157.00836563110352 

Best iterations: [153, 24, 153, 24, 82, 16, 89, 39, 119, 19, 92, 53, 89, 80, 111, 29, 91, 28, 90] 

Visualize model feature importance.
Predicting on test data.
Training model for column: identity_hate
(127657, 248) (31914, 248) (15

Predicting on test data.
Mean loss for current KFold run: 0.9883280389943511
Preparing submission.
